<img src="https://miro.medium.com/max/1400/1*zF3DzW57qD_LOGO1jQwdcg.png" width="600">

In [37]:
import numpy as np

Create a random individual. An individual is in this case one schedule.

In [38]:
class Schedule:
    def __init__(self, I, N):
        self.I = I  # Number of intervals
        self.N = N  # Number of patients
        self.x = np.zeros(self.I) # Empty schedule

    def create_random_schedule(self):
        for i in range(self.N):
            i = np.random.choice(self.I, 1)  # Interval ID
            self.x[i] = self.x[i] + 1
    def get_schedule(self): return(self.x)
    def print_schedule(self): print(self.x)

##########
## TEST ##
##########

I = 10  # Number of time slots
N = 8  # Number of patients
scheduleObject = Schedule(I, N)
scheduleObject.print_schedule()
scheduleObject.create_random_schedule()
schedule = scheduleObject.get_schedule()
schedule


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


array([0., 0., 2., 0., 2., 0., 1., 1., 2., 0.])

Create population of individuals

In [39]:
class Population:
    def __init__(self, I, N, size):
        self.I = I
        self.N = N
        self.size = size
        self.individuals = []
        self.__create_new_individuals()

    def create_new_individuals(self):
        for i in range(self.size):
            x = Schedule(self.I, self.N)
            x.create_random_schedule()
            x = x.get_schedule()
            self.individuals.append(x)
            
    __create_new_individuals = create_new_individuals
    
    def get_individuals(self): return(self.individuals)
    
##########
## TEST ##
##########

size = 12
populationObject = Population(I, N, size)
populationObject.individuals


[array([2., 1., 2., 1., 0., 0., 0., 1., 0., 1.]),
 array([0., 1., 1., 2., 0., 2., 0., 0., 0., 2.]),
 array([3., 1., 0., 0., 2., 0., 1., 0., 0., 1.]),
 array([1., 3., 0., 0., 0., 2., 0., 1., 0., 1.]),
 array([1., 2., 0., 0., 1., 2., 0., 0., 2., 0.]),
 array([1., 0., 0., 1., 1., 1., 2., 1., 1., 0.]),
 array([1., 0., 0., 0., 1., 1., 2., 1., 2., 0.]),
 array([2., 0., 0., 2., 0., 1., 1., 1., 0., 1.]),
 array([1., 0., 2., 0., 3., 0., 1., 0., 1., 0.]),
 array([0., 1., 1., 2., 1., 1., 1., 1., 0., 0.]),
 array([0., 1., 1., 1., 2., 1., 1., 0., 1., 0.]),
 array([1., 1., 1., 1., 0., 0., 2., 1., 1., 0.])]

<img src="crossover.png" width="600">

In [40]:
def crossover(parent1, parent2, I, n):
    child1 = parent1.copy()
    child2 = parent2.copy()
    
    # Calculate distribution for choosing intervals that will swap patients
    # The distribution is the normalized schedule. Intervals with zero patients will never be selected.
    norm1 = sum(child1)
    max1 = int(max(child1))
    n1 = min(n, max1)  # The number of swaps should not exceed the maximum number of patients in an interval. Otherwise it might happen that more patients are transfered than available.
    p1 = child1 / norm1
    
    # Swap patients
    i = np.random.choice(I, n1, p=p1)
    for i in i:
        child1[i] += -1
        child2[i] += 1
    
    # Calculate distribution for choosing intervals that will swap patients
    # The distribution is the normalized schedule. Intervals with zero patients will never be selected.
    norm2 = sum(child2)
    max2 = int(max(child2))
    n2 = min(n, max1)
    p2 = child2 / norm2
    j = np.random.choice(I, n2, p=p2)
    for j in j:
        child1[j] += 1
        child2[j] += -1
    return(child1, child2)

##########
## TEST ##
##########

population = populationObject.get_individuals()
parent1 = population[0]
parent2 = population[1]

print(parent1, parent2)
n = 4
children = crossover(parent1, parent2, I, n)
children


[2. 1. 2. 1. 0. 0. 0. 1. 0. 1.] [0. 1. 1. 2. 0. 2. 0. 0. 0. 2.]


(array([1., 1., 1., 2., 0., 1., 0., 1., 0., 1.]),
 array([1., 1., 2., 1., 0., 1., 0., 0., 0., 2.]))

In [52]:
def mutate(child, rate):
    if np.random.rand() < rate:
        x = len(child)
        i, j = np.random.choice(x, 2, replace=False).astype(int)
        a, b = child[i], child[j]
        child[i], child[j] = b, a
    return(child)

##########
## TEST ##
##########

child = children[0].copy()
print(child)
child = mutate(child, 1)
print(child)


[1. 1. 1. 2. 0. 1. 0. 1. 0. 1.]
[1. 1. 1. 0. 0. 1. 2. 1. 0. 1.]
